In [12]:
import xlrd

sheet = xlrd.open_workbook('parking_tickets_readme.xls').sheet_by_index(0)
for row in range(0,sheet.nrows):
    print(sheet.row(row))

[text:'PARKING TICKET DATA', empty:'']
[empty:'', empty:'']
[text:'ITEM NAME', text:'DESCRIPTION']
[empty:'', empty:'']
[text:'TAG_NUMBER_MASKED', text:'First three (3) characters masked with asterisks']
[text:'DATE_OF_INFRACTION', text:'Date the infraction occurred in YYYYMMDD format']
[text:'INFRACTION_CODE', text:'Applicable Infraction code (numeric)']
[text:'INFRACTION_DESCRIPTION', text:'Short description of the infraction']
[text:'SET_FINE_AMOUNT', text:'Amount of set fine applicable (in dollars)']
[text:'TIME_OF_INFRACTION', text:'Time the infraction occurred  in HHMM format (24-hr clock)']
[text:'LOCATION1', text:'Code to denote proximity (see table below)']
[text:'LOCATION2', text:'Street address']
[text:'LOCATION3', text:'Code to denote proximity (optional)']
[text:'LOCATION4', text:'Street address (optional)']
[text:'PROVINCE', text:'Province or state code of vehicle licence plate']
[empty:'', empty:'']
[empty:'', empty:'']
[text:'Proximity Code Table', empty:'']
[text:'PROX

Items to store in memory:
    -infraction_codes counters for all years
    -fine_amounts for all years
    -top locations for all years (numbers scrubbed)
    -number of tickets per day for all years (graph and raw data)

Predict 2016 parking tickets??!?!?!?

In [17]:
import csv
import re
years=['2008','2009','2010','2011','2012','2013','2014','2015']
files_to_open=['data/'+year+'.csv' for year in years]

def test_files(years, files_to_open):
    for file in files_to_open:
        try:
            with open(file,'rt',encoding='utf8') as f:
                reader = csv.DictReader(f)
                line = next(reader)
                print(collapse_address(line['location2']))
                print(file+" success")
        except FileNotFoundError:
            print(file+" failed")
            
test_files(years,files_to_open)

EAST MALL
data/2008.csv success
PRINCES BLVD
data/2009.csv success
DUPONT ST
data/2010.csv success
BRIAN HARRISON WAY
data/2011.csv success
PARKSIDE DR
data/2012.csv success
DANFORTH AVE
data/2013.csv success
SELLERS AVE
data/2014.csv success
CENTRE AVE
data/2015.csv success


In [18]:
import csv
import sys
from collections import defaultdict
from collections import Counter
import re

def collapse_address(address):
    return re.sub("^\d+\s|\s\d+\s|\s\d+$", "", address)

infraction_codes_dict={}
fine_amounts_dict={}
addresses_dict={}
infraction_codes = defaultdict(int)
fine_amounts = defaultdict(int)
addresses = defaultdict(int)
dates = defaultdict(int)
csv.field_size_limit(sys.maxsize)
years=['2008','2009','2010','2011','2012','2013','2014','2015']
files_to_open=['data/'+year+'.csv' for year in years]
for idx,file in enumerate(files_to_open):
    with open(file, 'rt', encoding='utf8') as f:
        reader = csv.DictReader(f)
        for line in reader:
            infraction_codes[line['infraction_code']]+=1
            fine_amounts[line['set_fine_amount']]+=1
            addresses[collapse_address(line['location2'])]+=1
            dates[line['date_of_infraction']]+=1
    infraction_codes_dict[years[idx]]=infraction_codes
    fine_amounts_dict[years[idx]]=fine_amounts
    addresses_dict[years[idx]]=addresses
    infraction_codes.clear()
    fine_amounts.clear()
    addresses.clear()
    print("PROCESSED "+years[idx])

PROCESSED 2008
PROCESSED 2009
PROCESSED 2010
PROCESSED 2011
PROCESSED 2012
PROCESSED 2013
PROCESSED 2014
PROCESSED 2015
